In [6]:
pip install selenium requests tqdm beautifulsoup4 webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [7]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo apt install -y ./google-chrome-stable_current_amd64.deb


--2025-03-08 04:26:43--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.179.136, 142.251.179.93, 142.251.179.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.179.136|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 114757600 (109M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.2’

google-chrome-stabl 100%[===================>] 109.44M   262MB/s    in 0.4s    

2025-03-08 04:26:44 (262 MB/s) - ‘google-chrome-stable_current_amd64.deb.2’ saved [114757600/114757600]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libauthen-sasl-perl libdata-dump-perl libencode-locale-perl
  libfile-basedir-perl libfile-desktopentry-perl libfile-listing-perl
  libfile-mimeinfo-perl libfont-afm-perl libfontenc1 libhtml-form-perl
  libhtml-format-perl libhtml-parser-perl libhtml-tagset-perl
  libhtml-tree-perl libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libio-html-perl
  libio-socket-ssl-perl libio-stringy-perl libip

In [8]:
pip install --upgrade webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import time
import requests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [10]:
!du -sh /teamspace/studios/this_studio/fbinaulimagedataset


du: cannot access '/teamspace/studios/this_studio/fbinaulimagedataset': No such file or directory


In [24]:
import os
import time
import re
import random
import requests
from tqdm import tqdm
from urllib.parse import unquote, urlparse, parse_qs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Configuration settings
CONFIG = {
    "wait_time": 5,  # Time to wait for page elements to load (seconds)
    "save_dir": "facebook_images_hd",  # Default save directory
    "failed_links_file": "failed_links.txt",  # File to save failed links
    "log_file": "scraper_log.txt",  # Log file for debugging
    "max_retries": 3,  # Maximum number of retries for failed downloads
    "retry_delay": 2,  # Initial delay between retries (seconds)
    "browser_restart_interval": 5,  # Restart browser every X posts
}

def log_message(message, file=CONFIG["log_file"]):
    """Log a message to both console and log file."""
    print(message)
    with open(file, "a") as f:
        f.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def setup_driver():
    """Setup and return a Chrome webdriver with appropriate options."""
    log_message("Setting up Chrome driver...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--window-size=1920,1080")
    
    # Use a realistic user agent
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59"
    ]
    options.add_argument(f"--user-agent={random.choice(user_agents)}")
    
    # Add more browser-like behavior
    options.add_argument("--disable-features=IsolateOrigins,site-per-process")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Set navigator.webdriver to undefined
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
        """
    })
    
    return driver

def rotate_browser_session(driver):
    """Rotate browser session to avoid detection."""
    try:
        # Clear cookies and cache
        driver.delete_all_cookies()
        driver.execute_script("window.localStorage.clear();")
        driver.execute_script("window.sessionStorage.clear();")
        
        # Randomize window size slightly
        width = random.randint(1024, 1920)
        height = random.randint(768, 1080)
        driver.set_window_size(width, height)
        
        # Change user agent periodically
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
        ]
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": random.choice(user_agents)})
        
        log_message("Browser session rotated")
    except Exception as e:
        log_message(f"Error rotating browser session: {str(e)}")

def create_session():
    """Create and return a requests session with more browser-like behavior."""
    session = requests.Session()
    
    # Use the appropriate retry parameter based on requests version
    try:
        retry_strategy = Retry(
            total=CONFIG["max_retries"],
            backoff_factor=CONFIG["retry_delay"],
            status_forcelist=[429, 500, 502, 503, 504],  # Don't retry on 403
            allowed_methods=["GET"]  # New parameter name
        )
    except TypeError:
        # Fall back to older parameter name if needed
        retry_strategy = Retry(
            total=CONFIG["max_retries"],
            backoff_factor=CONFIG["retry_delay"],
            status_forcelist=[429, 500, 502, 503, 504],  # Don't retry on 403
            method_whitelist=["GET"]  # Old parameter name
        )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    # Add common browser cookies and headers
    session.cookies.set("locale", "en_US")
    session.cookies.set("noscript", "1")
    session.headers.update({
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0"
    })
    
    return session

def clean_url(url):
    """Clean the URL while preserving necessary parameters."""
    # Some parameters are actually needed for the URL to work
    parsed_url = urlparse(url)
    params = parse_qs(parsed_url.query)
    
    # Only remove tracking parameters, keep those needed for access
    parameters_to_keep = ['_nc_cat', 'ccb', '_nc_sid', '_nc_ohc', '_nc_ht']
    clean_params = {k: v[0] for k, v in params.items() if k in parameters_to_keep}
    
    new_query = "&".join([f"{k}={v}" for k, v in clean_params.items()])
    
    # Construct the clean URL
    clean = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
    if new_query:
        clean += f"?{new_query}"
    
    return clean

def get_alternative_urls(url):
    """Generate alternative URL formats to try when the main URL fails."""
    alternatives = []
    
    # Try different CDN domains
    for cdn in ["scontent.xx", "scontent-iad3-1.xx", "scontent-iad3-2.xx", "scontent-dfw5-1.xx"]:
        alt = re.sub(r'scontent-[^\.]+\.xx', cdn, url)
        alternatives.append(alt)
    
    # Try with and without various parameters
    parsed = urlparse(url)
    params = parse_qs(parsed.query)
    
    # Generate URLs with different parameter combinations
    base = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
    
    # Just the base URL with minimal parameters
    if '_nc_cat' in params and '_nc_sid' in params:
        minimal = f"{base}?_nc_cat={params['_nc_cat'][0]}&_nc_sid={params['_nc_sid'][0]}"
        alternatives.append(minimal)
    
    # Try adding a direct download parameter
    alternatives.append(f"{url}&dl=1")
    
    return alternatives

def extract_image_urls_from_source(html_source):
    """Extract high-quality image URLs from HTML source."""
    soup = BeautifulSoup(html_source, "html.parser")
    image_urls = set()
    
    # Method 1: Find all images with scontent in src
    for img in soup.find_all("img"):
        if img.get("src") and "scontent" in img["src"]:
            url = clean_url(img["src"])
            image_urls.add(url)
    
    # Method 2: Look for preloaded image links
    for link in soup.find_all("link", {"rel": "preload", "as": "image"}):
        if link.get("href") and "scontent" in link["href"]:
            url = clean_url(link["href"])
            image_urls.add(url)
    
    # Method 3: Extract from meta tags
    for meta in soup.find_all("meta", {"property": "og:image"}):
        if meta.get("content") and "scontent" in meta["content"]:
            url = clean_url(meta["content"])
            image_urls.add(url)
    
    # Method 4: Look for data attributes with image URLs
    for element in soup.find_all(attrs={"data-visualcompletion": "media-vc-image"}):
        if element.get("src") and "scontent" in element["src"]:
            url = clean_url(element["src"])
            image_urls.add(url)
    
    # Method 5: Extract from background-image style
    for element in soup.find_all(style=True):
        if "background-image" in element["style"] and "scontent" in element["style"]:
            match = re.search(r'url\([\'"]?(.*?)[\'"]?\)', element["style"])
            if match:
                url = clean_url(match.group(1))
                if "scontent" in url:
                    image_urls.add(url)
    
    # Method 6: Extract from data-store attribute (common in Facebook)
    for element in soup.find_all(attrs={"data-store": True}):
        data_store = element.get("data-store")
        if data_store and "scontent" in data_store:
            urls = re.findall(r'https?://[^"\']+scontent[^"\']+', data_store)
            for url in urls:
                clean = clean_url(url)
                image_urls.add(clean)
    
    # Method 7: Look for AJAX responses with image data
    for script in soup.find_all("script"):
        if script.string and "scontent" in script.string:
            urls = re.findall(r'https?://[^"\']+scontent[^"\']+', script.string)
            for url in urls:
                if "jpg" in url.lower() or "jpeg" in url.lower() or "png" in url.lower():
                    clean = clean_url(url)
                    image_urls.add(clean)
    
    return list(image_urls)

def click_on_image(driver):
    """Try to click on images to get to the full-resolution version."""
    try:
        # Wait for and find clickable images using various selectors
        selectors = [
            "img[data-visualcompletion='media-vc-image']",
            "a[role='link'] img[src*='scontent']",
            "img[class*='x1bwycvy']",  # Common Facebook image class
            "img[class*='x168nmei']",
            "div[role='button'] img[src*='scontent']",
            "img[src*='scontent']"
        ]
        
        clicked = False
        for selector in selectors:
            try:
                # Wait for selector to be present
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                )
                
                # Find all matching elements
                images = driver.find_elements(By.CSS_SELECTOR, selector)
                
                if images:
                    # Try to click each image
                    for img in images:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", img)
                            time.sleep(0.5)  # Allow time for scrolling
                            
                            # Try different click methods
                            try:
                                img.click()  # Direct click
                            except:
                                try:
                                    driver.execute_script("arguments[0].click();", img)  # JavaScript click
                                except:
                                    # Try to find and click the parent element
                                    parent = driver.execute_script("return arguments[0].parentNode;", img)
                                    driver.execute_script("arguments[0].click();", parent)
                            
                            clicked = True
                            time.sleep(2)  # Wait for image to load in full resolution
                        except Exception as e:
                            log_message(f"Failed to click image using {selector}: {str(e)}")
                            continue
            except TimeoutException:
                continue
        
        return clicked
    except Exception as e:
        log_message(f"Error in click_on_image function: {str(e)}")
        return False

def get_high_quality_images(driver, post_url, timeout=60):
    """Extract high-quality images from a Facebook post with timeout."""
    log_message(f"Processing URL: {post_url}")
    
    # Set a page load timeout
    driver.set_page_load_timeout(timeout)
    
    try:
        # Navigate to the post
        log_message("Attempting to load page...")
        start_time = time.time()
        driver.get(post_url)
        load_time = time.time() - start_time
        log_message(f"Page loaded in {load_time:.2f} seconds")
        
        # Wait for page content
        log_message(f"Waiting {CONFIG['wait_time']} seconds for dynamic content...")
        time.sleep(CONFIG["wait_time"])
        
        # Get initial image URLs
        log_message("Extracting initial images...")
        initial_html = driver.page_source
        initial_images = extract_image_urls_from_source(initial_html)
        log_message(f"Found {len(initial_images)} images in initial page")
        
        # Try to click on images to get full resolution versions
        log_message("Attempting to click on images...")
        click_start = time.time()
        clicked = click_on_image(driver)
        click_time = time.time() - click_start
        
        if clicked:
            log_message(f"Successfully clicked on images in {click_time:.2f} seconds, extracting high-res versions")
            time.sleep(3)  # Wait longer for modal popup
            
            # Get image URLs after clicking
            log_message("Extracting images after clicking...")
            after_click_html = driver.page_source
            after_click_images = extract_image_urls_from_source(after_click_html)
            log_message(f"Found {len(after_click_images)} images after clicking")
        else:
            log_message(f"No images clicked after {click_time:.2f} seconds, using initial image URLs only")
            after_click_images = []
        
        # Combine and deduplicate the image URLs
        all_image_urls = list(set(initial_images + after_click_images))
        
        # Apply additional URL modifications to ensure highest resolution
        high_res_urls = []
        for url in all_image_urls:
            high_res_url = clean_url(url)
            high_res_urls.append(high_res_url)
        
        log_message(f"Found {len(high_res_urls)} high-quality images")
        return list(set(high_res_urls))  # Remove duplicates
    
    except Exception as e:
        log_message(f"Error processing {post_url}: {str(e)}")
        # Force continue even if there's an error
        driver.execute_script("window.stop();")
        return []

def download_image(session, url, folder, post_index, img_index):
    """Download an image with improved error handling."""
    try:
        # Prepare filename
        if "?" in url:
            base_name = url.split("?")[0].split("/")[-1]
        else:
            base_name = url.split("/")[-1]
        
        # Ensure the base name has a proper extension
        if not (base_name.lower().endswith('.jpg') or base_name.lower().endswith('.jpeg') or 
                base_name.lower().endswith('.png') or base_name.lower().endswith('.gif')):
            base_name += ".jpg"  # Default to jpg if no extension
        
        filename = os.path.join(folder, f"post{post_index}_img{img_index}_{base_name}")
        
        # Try the original URL first, then alternatives if needed
        urls_to_try = [url] + get_alternative_urls(url)
        
        for current_url_index, current_url in enumerate(urls_to_try):
            try:
                # Add a delay to avoid rate limiting
                time.sleep(random.uniform(1, 3))
                
                # Rotate user agents
                user_agents = [
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
                ]
                
                headers = {
                    "User-Agent": random.choice(user_agents),
                    "Referer": "https://www.facebook.com/",
                    "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
                    "Origin": "https://www.facebook.com",
                    "sec-ch-ua": '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"'
                }
                
                # Add randomness to headers to avoid detection
                if random.random() > 0.5:
                    headers["Accept-Encoding"] = "gzip, deflate, br"
                if random.random() > 0.5:
                    headers["sec-ch-ua-mobile"] = "?0"
                
                log_message(f"Trying URL {current_url_index + 1}/{len(urls_to_try)}: {current_url}")
                
                response = session.get(current_url, stream=True, headers=headers, timeout=15)
                
                if response.status_code == 200:
                    # Check content type to ensure it's an image
                    content_type = response.headers.get('Content-Type', '')
                    if not content_type.startswith('image/'):
                        log_message(f"Warning: Content-Type is {content_type}, not an image")
                        continue
                    
                    # Download the image
                    total_size = int(response.headers.get('content-length', 0))
                    with open(filename, "wb") as file, tqdm(
                        desc=f"Downloading {os.path.basename(filename)}",
                        total=total_size,
                        unit='B',
                        unit_scale=True,
                        unit_divisor=1024,
                    ) as bar:
                        for chunk in response.iter_content(chunk_size=1024):
                            size = file.write(chunk)
                            bar.update(size)
                    
                    # Verify file integrity
                    if os.path.exists(filename) and os.path.getsize(filename) > 1000:  # Basic size check
                        log_message(f"Successfully downloaded: {filename}")
                        return True
                    else:
                        log_message(f"Downloaded file too small, may be corrupted")
                        if os.path.exists(filename):
                            os.remove(filename)
                
                elif response.status_code == 403:
                    log_message(f"Access forbidden (403) for URL variant {current_url_index + 1}, trying next alternative")
                
                else:
                    log_message(f"HTTP status {response.status_code} for URL variant {current_url_index + 1}")
                
            except requests.exceptions.RequestException as req_err:
                log_message(f"Request error for URL variant {current_url_index + 1}: {str(req_err)}")
        
        log_message(f"Failed to download image after trying all URL alternatives")
        return False
        
    except Exception as e:
        log_message(f"Fatal error downloading image: {str(e)}")
        return False

def main():
    """Main function to run the Facebook image scraper."""
    # Clear log file
    open(CONFIG["log_file"], 'w').close()
    
    # Prompt user for input file
    file_path = input("Enter the full path of the .txt file containing Facebook post links: ")
    
    # Read links from file
    try:
        with open(file_path, "r") as file:
            fb_links = [line.strip() for line in file if line.strip().startswith("http")]
        log_message(f"Found {len(fb_links)} links in the file")
    except FileNotFoundError:
        log_message("File not found!")
        return
    
    # Create output directory
    os.makedirs(CONFIG["save_dir"], exist_ok=True)
    
    # Initialize driver and session
    driver = setup_driver()
    session = create_session()
    
    processed = 0
    failed_links = []
    
    # Process each link
    for i, link in enumerate(fb_links):
        try:
            # Restart browser periodically to avoid detection patterns
            if i > 0 and i % CONFIG["browser_restart_interval"] == 0:
                log_message("Restarting browser session...")
                driver.quit()
                time.sleep(5)  # Wait before restarting
                driver = setup_driver()
            else:
                # Just rotate the browser session for freshness
                rotate_browser_session(driver)
            
            processed += 1
            log_message(f"Processing {processed}/{len(fb_links)}: {link}")
            
            # Extract high-quality images
            high_res_imgs = get_high_quality_images(driver, link)
            
            if not high_res_imgs:
                log_message(f"No high-quality images found in {link}")
                failed_links.append(link)
                continue
            
            # Download each image
            successful_downloads = 0
            for j, img_url in enumerate(high_res_imgs):
                log_message(f"Downloading image {j+1}/{len(high_res_imgs)}")
                success = download_image(session, img_url, CONFIG["save_dir"], i+1, j+1)
                if success:
                    successful_downloads += 1
            
            log_message(f"Successfully downloaded {successful_downloads}/{len(high_res_imgs)} images from post {i+1}")
            
            # If no successful downloads, add to failed links
            if successful_downloads == 0 and len(high_res_imgs) > 0:
                failed_links.append(link)
            
            # Pause between posts with random delay to avoid being blocked
            pause_time = random.uniform(3, 8)
            log_message(f"Pausing for {pause_time:.2f} seconds before next post")
            time.sleep(pause_time)
            
        except Exception as e:
            log_message(f"Failed to process {link}: {str(e)}")
            failed_links.append(link)
    
    # Close the browser
    driver.quit()
    
    # Save failed links
    if failed_links:
        with open(CONFIG["failed_links_file"], "w") as f:
            for fl in failed_links:
                f.write(fl + "\n")
        log_message(f"Failed links saved in {CONFIG['failed_links_file']}")
    
    log_message("Process completed!")

if __name__ == "__main__":
    main()

Found 229 links in the file
Setting up Chrome driver...
Error rotating browser session: Message: <unknown>: Failed to read the 'localStorage' property from 'Window': Storage is disabled inside 'data:' URLs.
  (Session info: chrome=134.0.6998.35)
Stacktrace:
#0 0x55b6872bc46a <unknown>
#1 0x55b686d75ed0 <unknown>
#2 0x55b686d7ca1e <unknown>
#3 0x55b686d7f547 <unknown>
#4 0x55b686e14e2b <unknown>
#5 0x55b686ded862 <unknown>
#6 0x55b686e13ceb <unknown>
#7 0x55b686ded633 <unknown>
#8 0x55b686db91be <unknown>
#9 0x55b686dba981 <unknown>
#10 0x55b68728286b <unknown>
#11 0x55b68728673c <unknown>
#12 0x55b687269f12 <unknown>
#13 0x55b6872872b4 <unknown>
#14 0x55b68724e0af <unknown>
#15 0x55b6872aaad8 <unknown>
#16 0x55b6872aacb6 <unknown>
#17 0x55b6872bb2e6 <unknown>
#18 0x7f340034f609 start_thread

Processing 1/229: https://www.facebook.com/100079076260704/posts/643288114983725/?app=fbl
Processing URL: https://www.facebook.com/100079076260704/posts/643288114983725/?app=fbl
Attempting to load 

KeyboardInterrupt: 

usage: ipykernel_launcher.py [-h] [--url URL] [--urls-file URLS_FILE]
                             [--output OUTPUT] [--headless] [--login]
                             [--wait-time WAIT_TIME] [--post-count POST_COUNT]
ipykernel_launcher.py: error: unrecognized arguments: --f=/teamspace/studios/this_studio/.local/share/jupyter/runtime/kernel-v2-9645Gb8ozetmCwU.json


SystemExit: 2

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
#!/usr/bin/env python3
import os
import time
import sys
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    # (Optional) reduce logging
    chrome_options.add_argument("--log-level=3")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scrape_images(url, output_dir="images"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    driver = init_driver()
    driver.get(url)

    # Wait until at least one high-res image appears. Adjust the timeout as needed.
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img[data-visualcompletion='media-vc-image']"))
        )
    except Exception as e:
        print(f"Error waiting for images to load: {e}")
        driver.quit()
        return

    # Optionally scroll down to trigger lazy loading.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow extra time for images to load

    # Find all high-resolution image elements (the high-res version typically has this attribute)
    image_elements = driver.find_elements(By.CSS_SELECTOR, "img[data-visualcompletion='media-vc-image']")
    print(f"Found {len(image_elements)} image element(s).")

    downloaded = 0
    for index, img in enumerate(image_elements):
        src = img.get_attribute("src")
        if src:
            print(f"Downloading image {index+1}: {src}")
            try:
                img_data = requests.get(src).content
                filename = os.path.join(output_dir, f"image_{index+1}.jpg")
                with open(filename, "wb") as f:
                    f.write(img_data)
                downloaded += 1
            except Exception as e:
                print(f"Failed to download image {index+1}: {e}")

    driver.quit()
    print(f"Downloaded {downloaded} image(s) to '{output_dir}'.")

if __name__ == "__main__":
    if len(sys.argv) > 1:
        post_url = sys.argv[1]
        scrape_images(post_url)
    else:
        print("Usage: python Facebook_Image_Scraper.py <Facebook Post URL>")


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=134.0.6998.35)
Stacktrace:
#0 0x5638751c346a <unknown>
#1 0x563874c7cd23 <unknown>
#2 0x563874c6391c <unknown>
#3 0x563874c61f2d <unknown>
#4 0x563874c6257a <unknown>
#5 0x563874c80369 <unknown>
#6 0x563874d1b9f5 <unknown>
#7 0x563874cf4862 <unknown>
#8 0x563874d1aceb <unknown>
#9 0x563874cf4633 <unknown>
#10 0x563874cc01be <unknown>
#11 0x563874cc1981 <unknown>
#12 0x56387518986b <unknown>
#13 0x56387518d73c <unknown>
#14 0x563875170f12 <unknown>
#15 0x56387518e2b4 <unknown>
#16 0x5638751550af <unknown>
#17 0x5638751b1ad8 <unknown>
#18 0x5638751b1cb6 <unknown>
#19 0x5638751c22e6 <unknown>
#20 0x7f3f3f73b609 start_thread


In [ ]:
import os
import time
import json
import re
import argparse
from datetime import datetime
from urllib.parse import urlparse, parse_qs

import requests
from bs4 import BeautifulSoup
from PIL import Image as PILImage
from io import BytesIO
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException


class FacebookImageScraper:
    def __init__(self):
        self.output_folder = "Facebook Images Scraped"
        self.failed_links_file = "failed_links.txt"
        self.timeout = 30
        self.driver = None
        self.session = None
        self.use_login = False
        self.email = None
        self.password = None
        self.headless = False
        
        # Create output directory
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)
            print(f"Created output directory: {self.output_folder}")
        else:
            print(f"Using existing output directory: {self.output_folder}")
    
    def setup_driver(self):
        """Set up and return a Selenium WebDriver"""
        print("Setting up browser...")
        chrome_options = Options()
        if self.headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        
        self.driver = webdriver.Chrome(options=chrome_options)
        return self.driver
    
    def setup_session(self):
        """Set up and return a requests Session"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.facebook.com/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0',
        }
        
        self.session = requests.Session()
        self.session.headers.update(headers)
        return self.session
    
    def login_to_facebook(self):
        """Login to Facebook using Selenium"""
        if not self.use_login or not self.email or not self.password:
            print("Proceeding without login. Some content may not be accessible.")
            return False
        
        print("Logging into Facebook...")
        self.driver.get("https://www.facebook.com/")
        
        # Accept cookies if prompted
        try:
            cookie_button = WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(string(), 'Allow') or contains(string(), 'Accept')]"))
            )
            cookie_button.click()
        except (TimeoutException, NoSuchElementException):
            pass
            
        # Enter email
        try:
            email_field = WebDriverWait(self.driver, self.timeout).until(
                EC.presence_of_element_located((By.ID, "email"))
            )
            email_field.send_keys(self.email)
            
            # Enter password
            password_field = self.driver.find_element(By.ID, "pass")
            password_field.send_keys(self.password)
            
            # Click login
            login_button = self.driver.find_element(By.NAME, "login")
            login_button.click()
            
            # Wait for login to complete
            try:
                WebDriverWait(self.driver, self.timeout).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@role='navigation']"))
                )
                print("Login successful")
                
                # Pass the cookies from Selenium to Requests session
                selenium_cookies = self.driver.get_cookies()
                for cookie in selenium_cookies:
                    self.session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
                
                return True
            except TimeoutException:
                print("Login may have failed or page structure has changed. Continuing anyway...")
                return False
        except Exception as e:
            print(f"Error during login: {str(e)}")
            return False
    
    def get_links_from_file(self, file_path):
        """Extract Facebook links from a text file"""
        try:
            with open(file_path, 'r') as file:
                links = []
                for line in file:
                    line = line.strip()
                    if line and ('facebook.com' in line or 'fb.com' in line):
                        links.append(line)
                
                print(f"Found {len(links)} Facebook links in file")
                return links
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
            return []
    
    def extract_image_urls_selenium(self, post_url):
        """Extract image URLs from a Facebook post using Selenium"""
        if not self.driver:
            self.setup_driver()
            
        # Navigate to the post URL
        print(f"Navigating to: {post_url}")
        self.driver.get(post_url)
        time.sleep(5)  # Allow time for the page to load
        
        # First get the visible image links as a fallback
        visible_image_links = []
        try:
            # Multiple XPath patterns to find visible image links
            xpath_patterns = [
                "//a[contains(@href, '/photo/?fbid=')]",
                "//a[contains(@href, '/photo?fbid=')]",
                "//a[contains(@href, 'set=pcb.')]",
                "//div[@role='article']//a[.//img]"
            ]
            
            for pattern in xpath_patterns:
                elements = self.driver.find_elements(By.XPATH, pattern)
                for element in elements:
                    link = element.get_attribute('href')
                    if link and link not in visible_image_links and ('fbid=' in link or 'photo' in link):
                        visible_image_links.append(link)
            
            if visible_image_links:
                print(f"Found {len(visible_image_links)} visible image links")
        except Exception as e:
            print(f"Error finding visible image links: {str(e)}")
        
        # Primary approach: Click on the first image and navigate through all photos
        image_links = []
        viewer_opened = False
        
        try:
            # Multiple attempts to find clickable images
            clickable_selectors = [
                "//div[@role='article']//img[not(contains(@src, 'emoji')) and not(contains(@src, 'icon'))]",
                "//div[contains(@class, 'userContent')]//img[not(contains(@src, 'emoji'))]",
                "//div[contains(@class, 'story')]//img[not(contains(@src, 'emoji'))]",
                "//a[contains(@href, '/photo')]//img",
                "//div[@role='main']//img[not(contains(@src, 'emoji')) and not(contains(@src, 'icon')) and not(contains(@src, 'gif')) and not(contains(@src, 'png'))]"
            ]
            
            for selector in clickable_selectors:
                try:
                    images = self.driver.find_elements(By.XPATH, selector)
                    
                    if images:
                        print(f"Found {len(images)} potential clickable images")
                        # Try each image until one opens the viewer
                        for i, img in enumerate(images[:5]):  # Try first 5 images maximum
                            try:
                                self.driver.execute_script("arguments[0].scrollIntoView(true);", img)
                                time.sleep(1)
                                img.click()
                                time.sleep(3)  # Wait for viewer to open
                                
                                # Check if we're in a photo viewer
                                try:
                                    viewer_element = WebDriverWait(self.driver, 5).until(
                                        EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Photo viewer']"))
                                    )
                                    print("Photo viewer opened successfully")
                                    viewer_opened = True
                                    break
                                except:
                                    # Try another way to detect the viewer
                                    current_url = self.driver.current_url
                                    if 'photo' in current_url and 'fbid=' in current_url:
                                        print("Photo viewer detected through URL")
                                        viewer_opened = True
                                        break
                                    else:
                                        print("Click did not open photo viewer, trying to go back...")
                                        self.driver.back()
                                        time.sleep(2)
                            except Exception as e:
                                # Try to recover
                                try:
                                    self.driver.back()
                                    time.sleep(2)
                                except:
                                    pass
                                
                        if viewer_opened:
                            break
                except Exception as e:
                    pass
            
            # If we successfully opened the viewer, extract all images
            if viewer_opened:
                print("Navigating through all photos in the viewer...")
                
                # Record the first image URL
                current_url = self.driver.current_url
                if 'fbid=' in current_url and current_url not in image_links:
                    image_links.append(current_url)
                
                # Navigate through all photos
                photo_count = 1
                max_attempts = 30  # Set a reasonable maximum
                consecutive_failures = 0
                
                while photo_count < max_attempts and consecutive_failures < 3:
                    try:
                        # Find the next button using multiple possible selectors
                        next_selectors = [
                            "//div[@aria-label='Next photo']",
                            "//div[contains(@aria-label, 'Next')]",
                            "//div[@data-testid='chevron-right-overlay']",
                            "//div[contains(@class, 'next')]//i",
                            "//button[contains(@class, 'next')]"
                        ]
                        
                        next_button = None
                        for selector in next_selectors:
                            try:
                                elements = self.driver.find_elements(By.XPATH, selector)
                                if elements:
                                    next_button = elements[0]
                                    break
                            except:
                                continue
                        
                        if next_button:
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                            self.driver.execute_script("arguments[0].click();", next_button)
                            time.sleep(1.5)  # Wait for the next photo to load
                            
                            # Get the URL of this photo
                            current_url = self.driver.current_url
                            if 'fbid=' in current_url and current_url not in image_links:
                                image_links.append(current_url)
                                photo_count += 1
                                consecutive_failures = 0
                            else:
                                consecutive_failures += 1
                        else:
                            consecutive_failures += 1
                            
                    except Exception as e:
                        consecutive_failures += 1
                    
                    # If multiple failures, we've probably reached the end
                    if consecutive_failures >= 3:
                        break
                
                print(f"Found total of {len(image_links)} images through photo viewer")
                
                # Return to the post page
                try:
                    self.driver.execute_script("window.history.go(-{})".format(len(image_links)))
                    time.sleep(2)
                except:
                    # If history navigation fails, just go back to original URL
                    self.driver.get(post_url)
                    time.sleep(3)
        
        except Exception as e:
            print(f"Error in photo viewer navigation: {str(e)}")
        
        # If photo viewer approach failed, use the fallback visible links
        if not image_links and visible_image_links:
            print("Using fallback visible image links")
            image_links = visible_image_links
        
        # Clean up URLs - strip tracking parameters while keeping the essential fbid
        cleaned_links = []
        for url in image_links:
            try:
                # Extract the essential parts (fbid and set parameters)
                if 'fbid=' in url:
                    # Find fbid parameter
                    fbid_start = url.find('fbid=')
                    fbid_end = url.find('&', fbid_start)
                    if fbid_end == -1:
                        fbid_end = len(url)
                    fbid_param = url[fbid_start:fbid_end]
                    
                    # Try to find set parameter if it exists
                    set_param = ""
                    if 'set=' in url:
                        set_start = url.find('set=')
                        set_end = url.find('&', set_start)
                        if set_end == -1:
                            set_end = len(url)
                        set_param = "&" + url[set_start:set_end]
                    
                    # Create clean URL
                    clean_url = f"https://www.facebook.com/photo?{fbid_param}{set_param}"
                    if clean_url not in cleaned_links:
                        cleaned_links.append(clean_url)
                else:
                    # Just keep the original URL if we can't parse it properly
                    if url not in cleaned_links:
                        cleaned_links.append(url)
            except Exception as e:
                print(f"Error cleaning URL {url}: {str(e)}")
                if url not in cleaned_links:
                    cleaned_links.append(url)
        
        return cleaned_links
    
    def download_high_quality_images(self, image_urls, post_url):
        """Download high-quality images from a list of image URLs"""
        if not self.session:
            self.setup_session()
        
        downloaded_files = []
        failed_urls = []
        
        # Create a subfolder for this post
        parsed_url = urlparse(post_url)
        query_params = parse_qs(parsed_url.query)
        
        # Try to extract a meaningful folder name from the URL
        folder_name = datetime.now().strftime("%Y%m%d_%H%M%S")
        if 'fbid' in query_params:
            folder_name = f"post_{query_params['fbid'][0]}"
        elif parsed_url.path:
            clean_path = re.sub(r'[^\w\-_]', '_', parsed_url.path)
            folder_name = f"post_{clean_path[-20:]}"
            
        post_folder = os.path.join(self.output_folder, folder_name)
        if not os.path.exists(post_folder):
            os.makedirs(post_folder)
        
        # Use tqdm for a progress bar
        print(f"Downloading {len(image_urls)} images from post...")
        for i, url in enumerate(tqdm(image_urls, desc="Downloading images", unit="img")):
            try:
                # Get the image viewing page
                img_page_response = self.session.get(url)
                img_page_response.raise_for_status()
                
                # Parse the page to find the high-quality image
                soup = BeautifulSoup(img_page_response.text, 'html.parser')
                
                # Look for image URLs in the page source using regex
                image_pattern = r'https:\/\/scontent[^"\']+\.(?:jpg|jpeg|png|gif)'
                found_urls = re.findall(image_pattern, img_page_response.text)
                
                # Filter out small images, icons, etc.
                high_quality_urls = [u for u in found_urls if not ('icon' in u.lower() or 'emoji' in u.lower())]
                
                # Remove duplicates while preserving order
                high_quality_urls = list(dict.fromkeys(high_quality_urls))
                
                # If we found high-quality images, download the largest one
                if high_quality_urls:
                    # Generate a filename with index
                    filename = f"image_{i+1}.jpg"
                    file_path = os.path.join(post_folder, filename)
                    
                    # Find the largest image (often the first in the list)
                    best_url = high_quality_urls[0]
                    
                    # Try to get actual image size from headers
                    best_size = 0
                    for img_url in high_quality_urls[:3]:  # Check first few to save time
                        try:
                            head_response = self.session.head(img_url, timeout=5)
                            size = int(head_response.headers.get('Content-Length', 0))
                            if size > best_size:
                                best_size = size
                                best_url = img_url
                        except:
                            continue
                    
                    # Download the best image
                    img_response = self.session.get(best_url, stream=True)
                    img_response.raise_for_status()
                    
                    # Save the file
                    with open(file_path, 'wb') as f:
                        for chunk in img_response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                    
                    downloaded_files.append(file_path)
                else:
                    # Fall back to looking for image elements
                    images = soup.find_all('img', attrs={'data-visualcompletion': 'media-vc-image'})
                    if not images:
                        # Try any large image
                        images = [img for img in soup.find_all('img') 
                                 if img.get('width', '0').isdigit() and int(img.get('width', '0')) > 300]
                    
                    if images:
                        src = images[0].get('src')
                        # Clean the URL
                        clean_url = src.replace('&amp;', '&')
                        
                        # Generate a filename with index
                        filename = f"image_{i+1}.jpg"
                        file_path = os.path.join(post_folder, filename)
                        
                        # Download the image
                        img_response = self.session.get(clean_url, stream=True)
                        img_response.raise_for_status()
                        
                        # Save the file
                        with open(file_path, 'wb') as f:
                            for chunk in img_response.iter_content(chunk_size=8192):
                                if chunk:
                                    f.write(chunk)
                        
                        downloaded_files.append(file_path)
                    else:
                        failed_urls.append(url)
                        print(f"No high-quality image found for URL: {url}")
            except Exception as e:
                failed_urls.append(url)
                print(f"Error downloading image from {url}: {str(e)}")
        
        return downloaded_files, failed_urls
    
    def process_post(self, post_url):
        """Process a single Facebook post"""
        try:
            # Extract image URLs using Selenium
            image_urls = self.extract_image_urls_selenium(post_url)
            
            if not image_urls:
                print(f"No images found in post: {post_url}")
                return [], [post_url]
            
            # Download high-quality images
            downloaded_files, failed_urls = self.download_high_quality_images(image_urls, post_url)
            
            return downloaded_files, failed_urls
            
        except Exception as e:
            print(f"Error processing post {post_url}: {str(e)}")
            return [], [post_url]
    
    def process_links(self, links):
        """Process a list of Facebook post links"""
        all_downloaded_files = []
        all_failed_links = []
        
        # Set up the session and driver
        self.setup_session()
        self.setup_driver()
        
        # Login if needed
        if self.use_login:
            self.login_to_facebook()
        
        # Process each link with a progress bar
        print(f"Processing {len(links)} links...")
        for i, link in enumerate(tqdm(links, desc="Processing links", unit="link")):
            print(f"\nProcessing link {i+1}/{len(links)}: {link}")
            downloaded_files, failed_urls = self.process_post(link)
            
            all_downloaded_files.extend(downloaded_files)
            all_failed_links.extend(failed_urls)
            
            # Optional sleep to avoid rate limiting
            if i < len(links) - 1:
                time.sleep(2)
        
        # Log failed links
        if all_failed_links:
            with open(self.failed_links_file, 'w') as f:
                for link in all_failed_links:
                    f.write(f"{link}\n")
            print(f"Logged {len(all_failed_links)} failed links to {self.failed_links_file}")
        
        # Clean up
        if self.driver:
            self.driver.quit()
        
        return all_downloaded_files, all_failed_links
    
    def run(self):
        """Run the Facebook Image Scraper with user input"""
        print("=" * 50)
        print("Facebook Image Scraper")
        print("=" * 50)
        
        # Get input from user
        input_path = input("Enter a Facebook post URL or path to a .txt file containing links: ").strip()
        
        # Determine if it's a file or a direct URL
        links = []
        if input_path.endswith('.txt') and os.path.exists(input_path):
            links = self.get_links_from_file(input_path)
            if not links:
                print("No valid Facebook links found in the file.")
                return
        elif 'facebook.com' in input_path or 'fb.com' in input_path:
            links = [input_path]
        else:
            print("Invalid input. Please enter a valid Facebook URL or path to a .txt file.")
            return
        
        # Ask for login credentials
        self.use_login = input("Do you want to login to Facebook for better access? (y/n): ").lower() == 'y'
        if self.use_login:
            self.email = input("Enter your Facebook email: ")
            self.password = input("Enter your Facebook password: ")
        
        # Ask for headless mode
        self.headless = input("Run in headless mode (no visible browser)? (y/n): ").lower() == 'y'
        
        # Process the links
        start_time = time.time()
        downloaded_files, failed_links = self.process_links(links)
        
        # Print summary
        print("\n" + "=" * 50)
        print("Scraping Summary")
        print("=" * 50)
        print(f"Total links processed: {len(links)}")
        print(f"Total images downloaded: {len(downloaded_files)}")
        print(f"Failed links: {len(failed_links)}")
        print(f"Images saved to: {os.path.abspath(self.output_folder)}")
        print(f"Total time: {time.time() - start_time:.2f} seconds")
        
        if failed_links:
            print(f"Failed links saved to: {os.path.abspath(self.failed_links_file)}")
        
        print("\nDone!")


if __name__ == "__main__":
    scraper = FacebookImageScraper()
    scraper.run()